## C S 329E HW 7

# Naive Bayes 

## Michel Gonzalez

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [1]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [2]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
  
    # since the d varies bewtween two formats
    # 'D/M/YY' or 'DD/MM/YYYY', the longer fomrat
    # has a length of 10 so I split the formating fucntion
    # into the two cases
    if(len(d) < 10):
      
      standard = time.strptime(d, '%d/%m/%y')

    else:

      standard = time.strptime(d, '%d/%m/%Y')

    return standard


In [3]:
df_melb['Date'] = df_melb['Date'].apply(standardize_date)
df_melb['unixtime'] = df_melb['Date'].apply( lambda x : int(time.mktime(x)))
df_melb = df_melb.drop(columns="Date")

In [4]:
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,h,399000,8.7,3032,1,1.0,904,53.0,1985.0,1462579200
1,3,h,1241000,13.9,3165,1,1.0,643,NaN,NaN,1472342400
2,2,u,550000,3.0,3067,1,1.0,1521,NaN,NaN,1499472000
3,3,u,691000,8.4,3072,1,1.0,170,NaN,NaN,1498262400
4,2,u,657500,4.6,3122,1,1.0,728,73.0,1965.0,1479513600


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [5]:
dict_priors = {} #your code here

# gets the class labels and prioirs and 
# fills the dictionary with the labels
# and values
classes = df_melb[target_col].value_counts(normalize = True).index

values = df_melb[target_col].value_counts(normalize = True).values

for i in range(0, len(classes)):

  dict_priors[classes[i]] = values[i]

# show the priors
dict_priors


{'h': 0.452, 't': 0.13, 'u': 0.418}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [6]:
dict_nb_model = dict()
for target in dict_priors.keys():
    # your code here

    # creates the nested dictionary for each class
    dict_nb_model[target] = {}

    # creates data frames with only a specific class from
    # the training set
    alt_df = df_melb[df_melb[target_col] == target]

    for col in df_melb.columns:

      # skips the target column
      if col == target_col:

        continue

      else:

        # cals mean & sd and fills the dictionary
        # with tuples for every attribute for each class label

        mean = alt_df[col].mean(skipna = True)

        sd = alt_df[col].std(skipna = True)

        tup = (mean, sd)

        dict_nb_model[target][col] = tup

In [7]:
display(dict_nb_model)

{'h': {'Bathroom': (1.5619469026548674, 0.6720871086493075),
  'BuildingArea': (156.24339622641511, 54.62662837301434),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Distance': (12.086725663716809, 7.397501132737295),
  'Landsize': (932.9646017699115, 3830.7934157687164),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Price': (1189022.3451327435, 586296.5794417895),
  'Rooms': (3.269911504424779, 0.7258264201127756),
  'YearBuilt': (1954.900826446281, 32.461876347154686),
  'unixtime': (1485717578.761062, 13838562.050601462)},
 't': {'Bathroom': (1.8461538461538463, 0.565430401076506),
  'BuildingArea': (138.66666666666666, 53.498637054290135),
  'Car': (1.6923076923076923, 0.5280588545286915),
  'Distance': (10.766153846153845, 4.870455475462387),
  'Landsize': (268.18461538461537, 276.57700624711265),
  'Postcode': (3121.6153846153848, 100.01588816090862),
  'Price': (1000169.2307692308, 421822.5363389935),
  'Rooms': (2.9076923076923076, 0.6052653582075831),
  'Yea

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [8]:
def get_p( mu, sigma, x):
    # your code here

    p = (1 /(sigma * np.sqrt(2 * np.pi))) * np.exp((-.5 * (((x - mu)/sigma) ** 2)))
    
    return p

In [9]:
# Test it
p = get_p( 0, 2, 0.5)
p

0.19333405840142462

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [10]:
def nb_class( dict_priors, dict_nb_model, observation):
    # your code here

    # creates a list of the class labels
    classes = list(dict_priors.keys())
    
    # crates the list to hold the probability
    # from each class
    bayes_prob = []

    for target in dict_priors.keys():

      # gets the priors 
      p_class = dict_priors[target]

      # since the conditional probabilities are multiplied
      # we set the starting value to 1 
      atts_probs = 1

      # I use k to keep track of what observation
      # is being looked at
      k = 0

      for attribute in dict_nb_model[target].keys():

        # skips all nan values in the dataframe
        if (pd.isna(observation[k]) ==  True):

          k += 1

        else:
          
          # gets the conditionals probs by using the gaussian distribution 
          # and the means', sds', and observation value x from each attribute
          atts_probs *= get_p(dict_nb_model[target][attribute][0], dict_nb_model[target][attribute][1], observation[k])

          k += 1

      naive_prob = p_class * atts_probs

      bayes_prob.append(naive_prob)

    # returns a classification based on the max 'probability'
    # from naive bayes 'probabilities'
    # recall that these values are proportions not an
    # actual probability (for me)
    return classes[np.argmax(bayes_prob)]

## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [11]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date'].apply( lambda x : int(time.mktime(x)))
df_test = df_test.drop(columns="Date")

# gest the actual classification from
# the test set and drops the column
actual = df_test[target_col]

df_test = df_test.drop(columns = target_col)

In [12]:
predictions = []

for (indx,row) in df_test.iterrows():
  
  # your code here
  predictions.append(nb_class(dict_priors, dict_nb_model, row))

In [13]:
acc = sum(predictions == actual)/ len(predictions) # your code here

In [14]:
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 57.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [15]:
# Imputation training
dict_imputation = dict()
# your code here
for col in df_melb.columns:

    if (col == target_col):

      continue

    else:
      
      dict_imputation[col] = np.mean(df_melb[col])
        
# Imputation - apply on the test data
# your code here
df_melb = df_melb.fillna(value = dict_imputation)

df_test = df_test.fillna(value = dict_imputation)

# Seperate the attributes from the target_col
# your code here

df_X = df_melb.drop(columns = target_col)

s_y = df_melb[target_col]

In [16]:
gnb = GaussianNB()
# your code here

# gets the classification prediction based on using imputation
y_pred = gnb.fit(df_X, s_y).predict(df_test)

In [17]:
acc = sum(y_pred == actual)/ len(s_y) # your code here
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 7.40%


## ANSWER TO "do you think imputation hurt or helped the classifier?"
The classifier actually got severly hurt by applying imputation to both the test and traning data. There was just under a 50% difference in accuracy. 